In [2]:
import pandas as pd
df = pd.read_excel('default of credit card clients.xls', header=1, index_col=0)

In [3]:
#check first 5 rows
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
ID,,,,,,,,,,,,,,,,,,,,,
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [5]:
#check shape
df.shape

(30000, 24)

In [6]:
#clean dataset
#remove inconsistencies in the data for education and pay
def clean_dataset(df):
    df.loc[~df.EDUCATION.isin([1,2,3,4]), "EDUCATION"] = pd.np.nan
    for i in [0,2,3,4,5,6]:
        df.loc[df['PAY_{}'.format(i)]<-1] = pd.np.nan
    df.dropna(inplace=True)

clean_dataset(df)

In [7]:
#create dummy variables for categorical data
def process_categorical_features(df):
    dummies_education = pd.get_dummies(df.EDUCATION, prefix="EDUCATION", drop_first=True)
    dummies_marriage = pd.get_dummies(df.MARRIAGE, prefix="MARRIAGE", drop_first=True)
    df.drop(["EDUCATION", "MARRIAGE"], axis=1, inplace=True)
    return pd.concat([df, dummies_education, dummies_marriage], axis=1)
df = process_categorical_features(df)

In [8]:
#check head
df.head()

,LIMIT_BAL,SEX,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,...,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month,EDUCATION_2.0,EDUCATION_3.0,EDUCATION_4.0,MARRIAGE_1.0,MARRIAGE_2.0,MARRIAGE_3.0
ID,,,,,,,,,,,,,,,,,,,,,
2,120000.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,2.0,2682.0,...,1000.0,0.0,2000.0,1.0,1,0,0,0,1,0
3,90000.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,0.0,29239.0,...,1000.0,1000.0,5000.0,0.0,1,0,0,0,1,0
4,50000.0,2.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,46990.0,...,1100.0,1069.0,1000.0,0.0,1,0,0,1,0,0
5,50000.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,0.0,8617.0,...,9000.0,689.0,679.0,0.0,1,0,0,1,0,0
6,50000.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,64400.0,...,1000.0,1000.0,800.0,0.0,0,0,0,0,1,0


In [9]:
#get target and feature variables
y = df['default payment next month']
X = df[[col for col in df.columns if col!="default payment next month"]]

In [10]:
#get train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25, random_state=42)

In [11]:
#get shape of train and test data
print("Size of train dataset: {} rows".format(X_train.shape[0]))
print("Size of test dataset: {} rows".format(X_test.shape[0]))

Size of train dataset: 17391 rows
Size of test dataset: 5797 rows


In [ ]:
import xgboost as xgb
classifier = xgb.sklearn.XGBClassifier(nthread=-1, seed=42)

In [ ]:
#using default classifier
classifier.fit(X_train, y_train)

In [ ]:
XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=42, silent=True, subsample=1)

In [ ]:
#evaluation
predictions = classifier.predict(X_test)
pd.DataFrame(predictions, index=X_test.index, columns=['Predicted default']).head()
pd.DataFrame(y_test).head()
print("Model Accuracy: {:.2f}%".format(100*classifier.score(X_test, y_test)))